In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [110]:
# Importing the dataset
#dataset = pd.read_csv('full.csv', delimiter='|')
#X = dataset.iloc[:, :-1].values
#y = dataset.iloc[:, 3].values

def get_dataset(first_class = 'H'):
    classes = set()
    with open('full.csv', 'r', encoding="utf8") as file:
        for line in file:
            data = line.split('|')
            if len(data) == 7:
                data_classes = data[1].split(',')
                cl_set = set()
                for cl in data_classes:
                    c = cl.upper()
                    c = c.replace(' ', '')
                    a = re.search(r'([A-H])([0-9]{2})([A-Z])', c, re.M|re.I)
                    try:
                        if a.group(1) == first_class:
                            classes.add(a.group(1) + a.group(2) + a.group(3))
                    except:
                        pass
    
    classes_dict = {}
    for i in classes:
        classes_dict[i] = 0

    X = []
    y = []
    
    with open('full.csv', 'r', encoding="utf8") as file:
        for line in file:
            data = line.split('|')
            if len(data) == 7:
                data_classes = data[1].split(',')
                my_classes = classes_dict.copy()
                has_class = False
                for cl in data_classes:
                    c = cl.upper()
                    c = c.replace(' ', '')
                    a = re.search(r'([A-H])([0-9]{2})([A-Z])', c, re.M|re.I)
                    try:
                        if a.group(1) == first_class:
                            my_classes[a.group(1) + a.group(2) + a.group(3)] = 1
                            has_class = True
                    except:
                        pass
                if has_class:
                    X.append([data[2], data[3]])
                    y.append(list(my_classes.values()))
    
    return [X, y, classes]

In [111]:
dataset = get_dataset(first_class = 'H')

In [112]:
X = dataset[0]
y = dataset[1]

In [113]:
# Cleaning the texts
import re
import nltk
corpus_titulo = []
corpus_resumo = []
for i in range(len(X)):
    titulo = X[i][1].split()
    resumo = X[i][0].split()
    for j in range(len(titulo)):
        titulo[j] = re.sub('[^a-zA-Z]', '', titulo[j])
    for j in range(len(resumo)):
        resumo[j] = re.sub('[^a-zA-Z]', '', resumo[j])
    titulo = ' '.join(titulo)
    resumo = ' '.join(resumo)
    corpus_titulo.append(titulo)
    corpus_resumo.append(resumo)

In [114]:
corpus_resumo[0]

'Envoltorio para maquinas eletricas'

In [115]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = None, ngram_range=(1,1))
X_titulo = cv.fit_transform(corpus_titulo)

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = None, ngram_range=(1,1))
X_resumo = cv.fit_transform(corpus_resumo)

In [116]:
# svd
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50)
X_titulo = svd.fit_transform(X_titulo)

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50)
X_resumo = svd.fit_transform(X_resumo)

In [117]:
# concatena titulo e resumo
X = np.concatenate((X_titulo, X_resumo), axis=1)

In [118]:
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=None)

In [119]:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = None)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [120]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [132]:
y_pred[3]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [125]:
acc = accuracy_score(np.array(y_test), y_pred)

In [138]:
np.array([[0,1,1,0,0,1],[0,1,1,0,1,1]])

array([[0, 1, 1, 0, 0, 1],
       [0, 1, 1, 0, 1, 1]])

In [139]:
accuracy_score(np.array([[0,1,0,0,0,1],[0,1,1,0,1,1]]), np.array([[1,1,1,0,0,1],[0,1,1,0,0,1]]))

0.0